# 시계열 기본 분석

이 노트북은 시계열 데이터 분석의 기본적인 단계를 다룹니다.
시계열 데이터의 특성을 파악하고, 간단한 모델을 통해 예측의 기준점을 마련하는 것을 목표로 합니다.

## 사용 가이드
1. **데이터 설정**: `DATA_PATH`, `DATE_COL`, `TARGET_COL` 변수를 실제 데이터에 맞게 수정합니다.
2. **분석 파라미터**: `FREQUENCY`, `WINDOW_SIZE`, `DECOMPOSE_TYPE`, `DIFF_ORDER`, `TRAIN_SIZE`, `VALIDATION_SIZE` 등 분석 목적과 데이터 특성에 맞게 파라미터를 조정합니다.
3. **실행**: 노트북 셀을 순차적으로 실행하며 각 단계의 결과와 설명을 확인합니다.

## 분석 흐름
1. **라이브러리 임포트 및 설정**: 분석에 필요한 라이브러리를 불러오고 기본 설정을 수행합니다.
2. **데이터 로드 및 탐색**: 지정된 경로에서 데이터를 로드하고 기본적인 정보(형태, 타입, 통계량 등)를 확인합니다.
3. **시계열 데이터 전처리**: 날짜 컬럼을 인덱스로 설정하고, 타겟 변수를 선택합니다. 결측치를 처리하고 필요시 리샘플링을 수행합니다.
4. **시계열 데이터 시각화**: 원본 데이터, 이동 평균, 분포 등을 시각화하여 데이터의 패턴과 특성을 직관적으로 파악합니다.
5. **시계열 구성 요소 분해**: 시계열 데이터를 추세(Trend), 계절성(Seasonality), 잔차(Residual) 성분으로 분해하여 각 구성 요소의 특징을 분석합니다.
6. **정상성 확인 및 변환**: 시계열 데이터의 정상성 여부를 통계적 검정(ADF)과 시각적 방법(ACF/PACF 플롯)으로 확인하고, 비정상성 시계열의 경우 차분을 통해 정상성을 확보합니다.
7. **데이터 분할**: 모델 학습과 성능 평가를 위해 데이터를 시간 순서에 따라 훈련(Train)과 테스트(Test) 세트로 분할합니다.
8. **시계열 모델 선택 가이드**: 시계열 데이터의 특성에 따른 적합한 모델을 선택합니다.
9. **기본 시계열 모델**: 복잡한 모델을 적용하기 전에 간단한 기준(Baseline) 모델들의 성능을 확인합니다.
10. **모델 비교**: 여러 기본 모델들의 성능 지표(RMSE, MAE, R²)를 비교하여 어떤 단순 모델이 해당 데이터에 상대적으로 더 적합한지 평가합니다.
11. **모델 학습 및 평가**: 학습 데이터로 모델을 학습시키고, 테스트 데이터로 모델의 성능을 평가합니다.
12. 결론: 시계열 분석의 기본 단계를 수행하여 데이터의 특성을 파악하고, 적합한 모델을 선택하여 예측을 수행했습니다.

## 1. 사용자 입력 파라미터 설정
분석 대상 데이터와 파라미터를 사용자의 환경과 목적에 맞게 설정합니다.
- **필수 수정**: `DATA_PATH`, `DATE_COL`, `TARGET_COL`은 반드시 실제 데이터에 맞게 수정해야 합니다.
- **선택적 수정**: 데이터의 주기(`FREQUENCY`), 분석 기법(`WINDOW_SIZE`, `DECOMPOSE_TYPE`, `DIFF_ORDER`), 데이터 분할 비율(`TRAIN_SIZE`, `VALIDATION_SIZE`) 등은 필요에 따라 조정합니다.

In [ ]:
# ===== 필수 수정 파라미터 =====
# 데이터 파일 경로 (CSV 형식 권장)
DATA_PATH = "../data/raw/your_data.csv"

# 날짜 열 이름 (데이터프레임 내 날짜/시간 정보가 있는 열)
DATE_COL = "date"

# 타겟 열 이름 (분석하려는 변수가 있는 열)
TARGET_COL = "value"

# ===== 선택적 수정 파라미터 =====
# 시계열 주기
FREQUENCY = "D"  # D: 일별, M: 월별, Q: 분기별, Y: 연별

# 분석 파라미터
WINDOW_SIZE = 7                          # 이동평균 및 계절성 모델 윈도우 크기
DECOMPOSE_TYPE = "multiplicative"        # 시계열 분해 타입: 'additive'(가법), 'multiplicative'(승법)
DECOMPOSE_PERIOD = 7                       # 시계열 분해 주기 (데이터 특성에 맞게 설정, 예: 7일, 12개월)
DIFF_ORDER = 1                           # 차분 차수 (정상성 확보 목적)
TRAIN_SIZE = 0.8                         # 훈련 데이터 비율 (8:2 분할)

# 출력 경로
OUTPUT_DIR = '../plots'            # 시각화 결과 저장 경로
DATA_OUTPUT_DIR = '../data/processed'  # 전처리된 데이터 저장 경로

## 2. 필요 라이브러리 임포트
데이터 분석, 시계열 처리, 시각화, 모델링 등에 필요한 라이브러리를 불러옵니다.

In [ ]:
# 기본 데이터 처리 라이브러리
import numpy as np
import pandas as pd
import warnings
from scipy import stats

# 시각화 라이브러리 및 설정
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
# 한글 폰트가 필요한 경우 아래 코드 활성화
# plt.rc('font', family='Malgun Gothic')
# plt.rcParams['axes.unicode_minus'] = False

# 시계열 분석 라이브러리
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# 머신러닝 및 평가 라이브러리
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

# 경고 메시지 비활성화
warnings.filterwarnings('ignore')

## 3. 데이터 로드 및 탐색
지정된 경로에서 데이터를 로드하고, 데이터의 기본적인 구조와 통계 정보를 확인하여 데이터에 대한 초기 이해를 돕습니다.

In [ ]:
# 데이터 로드
df = pd.read_csv(DATA_PATH)
print(f"데이터 로드 완료. 형태: {df.shape}")

# 날짜 열 처리
df[DATE_COL] = pd.to_datetime(df[DATE_COL])
print(f"'{DATE_COL}' 열을 날짜 형식으로 변환했습니다.")

# 데이터 정보 출력
print("\n데이터 기본 정보:")
print(df.info())
print("\n기술 통계량:")
print(df.describe())

# 처음 몇 행 확인
print("\n처음 5개 행:")
print(df.head())

## 4. 시계열 데이터 전처리
시계열 분석에 적합하도록 데이터를 가공합니다.
- **날짜 인덱스 설정**: 시간 순서를 기준으로 데이터를 분석하기 위해 날짜/시간 컬럼을 데이터프레임의 인덱스로 설정합니다.
- **타겟 변수 선택**: 분석하고 예측하려는 주요 변수(컬럼)를 선택합니다.
- **결측치 처리**: 데이터의 누락된 부분을 적절한 방법(여기서는 선형 보간)으로 채워 분석 오류를 방지합니다.
- **리샘플링 (선택적)**: 데이터의 시간 간격(주기)이 불규칙하거나 분석 목적에 맞는 주기로 변경해야 할 경우 사용합니다. (예: 일별 데이터를 월별 평균으로)

In [ ]:
# 날짜 인덱스로 설정
df.set_index(DATE_COL, inplace=True)
print(f"'{DATE_COL}'를 인덱스로 설정했습니다.")

# 타겟 변수 선택
ts_data = df[TARGET_COL].copy()
print(f"타겟 변수: {TARGET_COL}, 데이터 형태: {ts_data.shape}")

# 결측치 확인 및 처리
missing_count = ts_data.isnull().sum()
print(f"결측치 개수: {missing_count}")

### 4.1 결측치 처리
시계열 데이터에서 결측치는 분석 결과에 큰 영향을 미칠 수 있습니다. 결측치를 처리하는 여러 방법이 있습니다:

**시나리오 A: 선형 보간법 + 앞/뒤 값 채우기 (활성화됨)**
- **목적**: 시계열의 시간적 연속성을 최대한 유지하면서 결측치를 채웁니다.
- **적용 상황**: 대부분의 시계열 분석에서 권장되는 기본적인 접근법입니다.
- **장점**: 시간에 따른 데이터의 변화 패턴을 고려하여 자연스럽게 결측치를 보완합니다.
- **단점**: 급격한 변화나 이상치가 있는 구간에서는 정확도가 떨어질 수 있습니다.

In [ ]:
# 시나리오 A: 선형 보간법 + 앞/뒤 값 채우기 (활성화됨)
# 중간 결측치는 선형 보간법으로 채우기
ts_data = ts_data.interpolate(method='linear')

# 시작/끝 부분의 결측치는 인접 값으로 채우기
ts_data = ts_data.fillna(method='bfill').fillna(method='ffill')
print("결측치를 선형 보간법으로 처리했습니다.")
print(f"처리 후 결측치 개수: {ts_data.isnull().sum()}")

# **시나리오 B: 평균값 대체 (주석 처리됨)**
# # 목적: 간단한 방법으로 결측치를 처리합니다.
# # 적용 상황: 결측 패턴이 무작위적이고 결측률이 낮을 때 사용할 수 있습니다.
# # 장점: 구현이 간단하고 계산이 빠릅니다.
# # 단점: 시계열의 시간적 연속성을 고려하지 않아 계절성이나 추세를 왜곡할 수 있습니다.
# # ts_data = ts_data.fillna(ts_data.mean())
# # print("결측치를 평균값으로 대체했습니다.")
# # print(f"처리 후 결측치 개수: {ts_data.isnull().sum()}")

# **시나리오 C: 고급 방법 - Forward Fill 후 Backward Fill 조합 (주석 처리됨)**
# # 목적: 단순하면서도 시간적 순서를 일부 고려하는 방법입니다.
# # 적용 상황: 데이터가 안정적이거나 짧은 기간의 결측값이 있을 때 적합합니다.
# # 장점: 구현이 간단하며 시간적 연속성을 일부 보존합니다.
# # 단점: 긴 결측 구간에서는 동일한 값이 반복되어 패턴을 왜곡할 수 있습니다.
# # ts_data = ts_data.fillna(method='ffill').fillna(method='bfill')
# # print("결측치를 forward/backward fill로 처리했습니다.")
# # print(f"처리 후 결측치 개수: {ts_data.isnull().sum()}")

### 4.2 리샘플링
시계열 데이터는 종종 불규칙한 간격으로 수집되거나, 분석 목적에 맞게 다른 주기로 변환해야 할 수 있습니다.

**시나리오 A: 지정된 빈도로 리샘플링 (활성화됨)**
- **목적**: 데이터의 시간적 구조를 표준화하고 일관된 간격으로 만듭니다.
- **적용 상황**: 불규칙한 시간 간격의 데이터를 분석하거나, 분석에 적합한 다른 시간 단위로 변환할 때 사용합니다.
- **장점**: 일정한 시간 간격의 데이터는 시계열 분석 및 시각화에 더 적합합니다.
- **단점**: 원본 데이터의 세부적인 패턴이 손실될 수 있으며, 리샘플링 방식에 따라 결과가 달라질 수 있습니다.

In [ ]:
# 시나리오 A: 지정된 빈도로 리샘플링 (활성화됨)
# 주어진 빈도로 리샘플링
ts_resampled = ts_data.resample(FREQUENCY).mean()

# 리샘플링 후 결측치 처리
ts_resampled = ts_resampled.interpolate(method='linear')
ts_resampled = ts_resampled.fillna(method='bfill').fillna(method='ffill')

print(f"데이터를 '{FREQUENCY}' 빈도로 리샘플링했습니다.")
print(f"리샘플링 후 데이터 형태: {ts_resampled.shape}")
ts_data = ts_resampled

# **시나리오 B: 리샘플링 없이 원본 데이터 사용 (주석 처리됨)**
# # 목적: 원본 데이터의 정확한 시간 간격과 특성을 유지합니다.
# # 적용 상황: 데이터가 이미 일정한 간격으로 수집되었거나, 원본 데이터의 정확한 타임스탬프가 중요한 경우 사용합니다.
# # 장점: 원본 데이터의 모든 정보와 패턴이 보존됩니다.
# # 단점: 불규칙한 간격은 일부 시계열 분석 기법을 적용하기 어렵게 만들 수 있습니다.
# # print("원본 데이터를 그대로 사용합니다 (리샘플링 없음).")
# # print(f"데이터 형태: {ts_data.shape}")

# **시나리오 C: 다운샘플링/업샘플링 조합 (주석 처리됨)**
# # 목적: 다양한 시간 척도에서 데이터 패턴을 분석합니다.
# # 적용 상황: 복잡한 시계열 패턴을 다양한 시간 단위에서 파악하고자 할 때 사용합니다.
# # 장점: 여러 시간 척도의 패턴을 동시에 분석할 수 있습니다.
# # 단점: 구현이 복잡하고, 각 단계에서 데이터 왜곡이 발생할 수 있습니다.
# # # 1. 먼저 일별 데이터로 다운샘플링
# # daily_data = ts_data.resample('D').mean().interpolate()
# # # 2. 주별 집계 계산
# # weekly_agg = daily_data.resample('W').mean()
# # # 3. 다시 일별 데이터로 업샘플링하여 주간 패턴 파악
# # weekly_pattern = weekly_agg.resample('D').interpolate(method='cubic')
# # 
# # print("다운샘플링/업샘플링 조합 처리 완료")
# # print(f"최종 데이터 형태: {weekly_pattern.shape}")
# # ts_data = daily_data  # 일별 데이터를 기본으로 사용

## 5. 시계열 데이터 시각화
데이터를 시각화하여 추세, 계절성, 주기성, 이상치 등의 패턴을 직관적으로 파악합니다.
- **선 그래프**: 시간의 흐름에 따른 데이터 변화를 보여줍니다.
- **이동 평균**: 데이터의 단기 변동성을 완화하여 장기적인 추세를 파악하는 데 도움을 줍니다.
- **분포 플롯 (히스토그램, Q-Q 플롯)**: 데이터 값의 분포 형태와 정규성 가정을 확인합니다.

In [ ]:
# 시계열 데이터 시각화
plt.figure(figsize=(14, 7))
plt.plot(ts_data.index, ts_data.values, marker='o', markersize=3, linestyle='-', alpha=0.7)
plt.title(f'{TARGET_COL} 시계열 데이터')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_시계열_원본.png')
plt.show()

# 이동 평균 추가
plt.figure(figsize=(14, 7))
plt.plot(ts_data.index, ts_data.values, label='원본 데이터', alpha=0.5)
plt.plot(ts_data.rolling(window=WINDOW_SIZE).mean().index, 
         ts_data.rolling(window=WINDOW_SIZE).mean().values, 
         label=f'{WINDOW_SIZE}일 이동평균', linewidth=2)
plt.title(f'{TARGET_COL} 시계열 데이터와 이동평균')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_시계열_이동평균.png')
plt.show()

# 분포 확인
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
sns.histplot(ts_data, kde=True)
plt.title(f'{TARGET_COL} 분포')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
stats.probplot(ts_data, plot=plt)
plt.title('Q-Q 플롯')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_시계열_분포.png')
plt.show()

## 6. 시계열 구성 요소 분해
시계열 데이터를 눈에 보이지 않는 여러 구성 요소로 분해하여 데이터의 구조적 특징을 파악합니다.

**시계열의 주요 구성 요소**:
- **추세(Trend)**: 장기적인 데이터의 증가 또는 감소 경향
- **계절성(Seasonality)**: 특정 시간 주기로 반복되는 패턴 (예: 요일별, 월별, 연간)
- **잔차(Residual)**: 추세와 계절성을 제외한 불규칙한 변동 성분

분해 모델에는 두 가지 유형이 있습니다:
- **가법(Additive) 모델**: 구성 요소들이 더해져서 원본 데이터를 구성 (변동 폭이 일정할 때 적합)
- **승법(Multiplicative) 모델**: 구성 요소들이 곱해져서 원본 데이터를 구성 (변동 폭이 시간에 따라 증가할 때 적합)

현재 `DECOMPOSE_TYPE = "multiplicative"` (승법 모델)로 설정되어 있습니다.
데이터 특성에 맞게 파라미터 설정 섹션에서 `DECOMPOSE_TYPE`을 "additive" 또는 "multiplicative"로 변경할 수 있습니다.

In [ ]:
# 시계열 구성 요소 분해
# 참고: seasonal_decompose는 데이터 길이가 period의 2배 이상이어야 정상 작동합니다.
decomposition = seasonal_decompose(ts_data, model=DECOMPOSE_TYPE, period=DECOMPOSE_PERIOD)

# 분해 결과 시각화
plt.figure(figsize=(14, 12))

plt.subplot(4, 1, 1)
plt.plot(decomposition.observed)
plt.title('원본 데이터')
plt.grid(True, alpha=0.3)

plt.subplot(4, 1, 2)
plt.plot(decomposition.trend)
plt.title('추세 (Trend)')
plt.grid(True, alpha=0.3)

plt.subplot(4, 1, 3)
plt.plot(decomposition.seasonal)
plt.title('계절성 (Seasonality)')
plt.grid(True, alpha=0.3)

plt.subplot(4, 1, 4)
plt.plot(decomposition.resid)
plt.title('잔차 (Residual)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_시계열_분해.png')
plt.show()

# 구성 요소 저장 (추후 분석 활용 가능)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# 분해 과정에서 발생할 수 있는 결측치 처리
trend = trend.fillna(method='bfill').fillna(method='ffill')
seasonal = seasonal.fillna(method='bfill').fillna(method='ffill')
residual = residual.fillna(method='bfill').fillna(method='ffill')

print("시계열 구성 요소 분해 완료")

## 7. 정상성 확인 및 변환
많은 시계열 모델(특히 ARIMA)은 데이터가 정상성(Stationarity)을 만족한다고 가정합니다. 정상성이란 시계열의 통계적 특성(평균, 분산, 자기공분산)이 시간에 따라 변하지 않는 것을 의미합니다.

**정상성 확인 방법**:
- **시각적 방법**: 
  - 원본 시계열 플롯: 추세나 계절성이 없고 평균과 분산이 일정하게 보이는지 확인
  - ACF/PACF 플롯: 자기상관함수(ACF)와 부분자기상관함수(PACF) 플롯을 통해 자기상관 구조를 파악. ACF가 천천히 감소하면 비정상성을 의심할 수 있습니다.

- **통계적 방법**:
  - ADF 검정 (Augmented Dickey-Fuller Test): 귀무가설은 "단위근이 존재한다(=시계열이 비정상적이다)"이며, p-value가 유의수준(보통 0.05)보다 작으면 귀무가설을 기각하여 정상성을 만족한다고 볼 수 있습니다.
  - KPSS 검정: ADF 검정의 보완으로 사용할 수 있으며, 귀무가설이 "시계열이 정상적이다"로 ADF와 반대입니다.

**정상성 변환 방법**:
- **차분(Differencing)**: 가장 일반적인 방법으로, 현재 시점의 데이터에서 이전 시점의 데이터를 빼서 추세를 제거합니다.
- **로그 변환**: 분산이 증가하는 시계열에 적용하여 분산을 안정화시킵니다.
- **계절 차분**: 계절성을 제거하기 위해 특정 계절 주기(예: 12개월)만큼 떨어진 값과의 차이를 계산합니다.

In [ ]:
# ACF 및 PACF 플롯
plt.figure(figsize=(14, 7))

plt.subplot(1, 2, 1)
plot_acf(ts_data, lags=20, alpha=0.05, ax=plt.gca())
plt.title('자기상관함수 (ACF)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plot_pacf(ts_data, lags=20, alpha=0.05, method='ywm', ax=plt.gca())
plt.title('부분 자기상관함수 (PACF)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_ACF_PACF.png')
plt.show()

### 7.1 정상성 검정
시계열 데이터가 정상성을 만족하는지 통계적으로 검정합니다. ADF 검정에서 귀무가설은 "시계열이 비정상적이다"이며, 
p-value가 0.05보다 작으면 귀무가설을 기각하고 정상성을 만족한다고 판단합니다.

In [ ]:
# ADF 검정 실행
adf_result = adfuller(ts_data.dropna())
print('ADF 검정 결과:')
print(f'ADF 통계량: {adf_result[0]:.4f}')
print(f'p-value: {adf_result[1]:.4f}')
print('임계값:')
for key, value in adf_result[4].items():
    print(f'\t{key}: {value:.4f}')

# 결과 해석 (p-value 기준)
print('\nADF 검정 결과 해석:')
if adf_result[1] < 0.05:
    print('귀무가설 기각 (p < 0.05): 시계열이 정상적일 가능성이 높습니다.')
else:
    print('귀무가설 채택 (p >= 0.05): 시계열이 비정상적일 가능성이 높습니다.')

### 7.2 차분을 통한 정상성 확보
데이터가 비정상적일 경우, 차분을 적용하여 정상성을 확보할 수 있습니다. 차분은 현재 값에서 이전 값을 빼는 방식으로,
추세를 제거하는 효과가 있습니다. `DIFF_ORDER` 파라미터로 차분 차수를 조절합니다.

**차분의 종류**:
- **1차 차분**: 인접한 두 시점 간의 차이를 계산 (추세 제거에 효과적)
- **2차 차분**: 1차 차분을 한 번 더 적용 (강한 비선형 추세가 있을 때 사용)
- **계절 차분**: 계절 주기만큼 떨어진 값들의 차이를 계산 (계절성 제거에 효과적)

In [ ]:
# 차분 적용
diff_data = ts_data.diff(DIFF_ORDER).dropna()
print(f"\n{DIFF_ORDER}차 차분 데이터 생성 완료")

# 변환된 데이터 시각화
plt.figure(figsize=(14, 10))

plt.subplot(2, 1, 1)
plt.plot(ts_data, label='원본 데이터')
plt.title('원본 시계열 데이터')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(diff_data, label=f'{DIFF_ORDER}차 차분')
plt.title(f'{DIFF_ORDER}차 차분 데이터')
plt.xlabel('날짜')
plt.ylabel(f'차분({TARGET_COL})')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_시계열_차분.png')
plt.show()

# 차분 후 정상성 검정
adf_diff_result = adfuller(diff_data.dropna())
print(f'\n{DIFF_ORDER}차 차분 후 ADF 검정 결과:')
print(f'ADF 통계량: {adf_diff_result[0]:.4f}')
print(f'p-value: {adf_diff_result[1]:.4f}')

# 차분 후 결과 해석
if adf_diff_result[1] < 0.05:
    print('차분 후 시계열이 정상적일 가능성이 높습니다 (p < 0.05).')
else:
    print('차분 후에도 시계열이 비정상적일 가능성이 높습니다 (p >= 0.05). 추가 차분이 필요할 수 있습니다.')

## 8. 데이터 분할
모델 학습과 성능 평가를 위해 데이터를 시간 순서에 따라 훈련(Train)과 테스트(Test) 세트로 분할합니다.
- **훈련 세트**: 모델 학습에 사용됩니다.
- **테스트 세트**: 학습된 최종 모델의 일반화 성능을 평가하는 데 사용됩니다.
시계열 데이터는 과거 데이터로 미래를 예측하는 것이므로, 반드시 시간 순서를 유지하며 분할해야 합니다.

In [ ]:
# 훈련/테스트 분할 (8:2 비율)
train_test_split_idx = int(len(ts_data) * TRAIN_SIZE)

train_data = ts_data[:train_test_split_idx]
test_data = ts_data[train_test_split_idx:]

print(f"훈련 데이터: {train_data.shape}, 기간: {train_data.index.min()} ~ {train_data.index.max()}")
print(f"테스트 데이터: {test_data.shape}, 기간: {test_data.index.min()} ~ {test_data.index.max()}")

# 훈련/테스트 데이터 시각화
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data.values, label='훈련 데이터', color='blue')
plt.plot(test_data.index, test_data.values, label='테스트 데이터', color='red')
plt.axvline(x=train_data.index[-1], color='gray', linestyle='--')
plt.title('훈련/테스트 데이터 분할')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_훈련테스트분할.png')
plt.show()

## 9. 시계열 모델 선택 가이드

**시계열 데이터 특성에 따른 모델 선택**:

1. **추세만 있는 경우**:
   - 지수평활법(단순) 또는 ARIMA(p,1,q) 추천

2. **계절성만 있는 경우**:
   - 계절성 지수평활법 또는 SARIMA 추천

3. **추세와 계절성 모두 있는 경우**:
   - 삼중 지수평활법(Holt-Winters) 또는 SARIMA 추천

4. **복잡한 패턴이 있는 경우**:
   - Prophet 또는 딥러닝 모델(LSTM, GRU) 추천

5. **외부 변수의 영향이 큰 경우**:
   - SARIMAX 또는 딥러닝 모델 추천

## 10. 기본 시계열 모델
복잡한 모델을 적용하기 전에 간단한 기준(Baseline) 모델들의 성능을 확인합니다. 이는 향후 개발할 모델의 성능 향상 정도를 비교하는 척도가 됩니다.

**기본 시계열 모델 유형**:
- **추세 없음 (Naive)**: 마지막 관측값을 그대로 사용하여 미래를 예측합니다. 데이터 변화가 거의 없을 때 좋은 기준 성능을 제공합니다.
- **선형 추세 (Linear Trend)**: 시간에 따라 데이터가 선형적으로 증가하거나 감소한다고 가정하고 회귀 모델을 적용합니다.
- **이동 평균 (Moving Average)**: 과거 일정 기간(윈도우) 데이터의 평균값으로 미래를 예측합니다. 단기 변동성을 완화하는 효과가 있습니다.
- **계절성 (Seasonal Naive)**: 과거의 동일한 계절(주기)에 해당하는 값을 사용하여 미래를 예측합니다. 데이터에 뚜렷한 계절적 패턴이 있을 때 유용합니다.

이러한 간단한 모델들은 더 복잡한 모델의 성능을 평가하는 기준점을 제공하며, 데이터의 특성에 따라 때로는 복잡한 모델보다 더 나은 성능을 보일 수도 있습니다.

In [ ]:
# 추세 없음 모델 (최근 관측값 반복)
naive_predictions = pd.Series(index=test_data.index, data=train_data.iloc[-1])

# 시각화
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data.values, label='훈련 데이터', color='blue')
plt.plot(test_data.index, test_data.values, label='실제 테스트 데이터', color='red')
plt.plot(naive_predictions.index, naive_predictions.values, label='추세 없음 예측', color='green', linestyle='--')
plt.title('추세 없음 모델 (No Trend)')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_추세없음모델.png')
plt.show()

# 모델 평가
naive_mse = mean_squared_error(test_data, naive_predictions)
naive_rmse = np.sqrt(naive_mse)
naive_mae = mean_absolute_error(test_data, naive_predictions)
naive_r2 = r2_score(test_data, naive_predictions)

print("추세 없음 모델 평가:")
print(f"RMSE: {naive_rmse:.4f}")
print(f"MAE: {naive_mae:.4f}")
print(f"R²: {naive_r2:.4f}")

### 선형 추세 모델

In [ ]:
# 선형 추세 모델
X = np.arange(len(train_data)).reshape(-1, 1)
y = train_data.values

# 선형 회귀 모델 학습
linear_model = LinearRegression()
linear_model.fit(X, y)

# 테스트 데이터에 대한 예측
X_test = np.arange(len(train_data), len(train_data) + len(test_data)).reshape(-1, 1)
linear_predictions = pd.Series(linear_model.predict(X_test), index=test_data.index)

# 시각화
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data.values, label='훈련 데이터', color='blue')
plt.plot(test_data.index, test_data.values, label='실제 테스트 데이터', color='red')
plt.plot(linear_predictions.index, linear_predictions.values, label='선형 추세 예측', color='green', linestyle='--')
plt.title('선형 추세 모델')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_선형추세모델.png')
plt.show()

# 모델 평가
linear_mse = mean_squared_error(test_data, linear_predictions)
linear_rmse = np.sqrt(linear_mse)
linear_mae = mean_absolute_error(test_data, linear_predictions)
linear_r2 = r2_score(test_data, linear_predictions)

print("선형 추세 모델 평가:")
print(f"RMSE: {linear_rmse:.4f}")
print(f"MAE: {linear_mae:.4f}")
print(f"R²: {linear_r2:.4f}")

### 이동 평균 모델

In [ ]:
# 이동평균 모델
# 훈련 데이터의 마지막 윈도우 평균으로 테스트 기간 전체를 예측
ma_predictions = pd.Series(
    data=train_data.rolling(window=WINDOW_SIZE).mean().iloc[-1],
    index=test_data.index
)

# 시각화
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data.values, label='훈련 데이터', color='blue')
plt.plot(test_data.index, test_data.values, label='실제 테스트 데이터', color='red')
plt.plot(ma_predictions.index, ma_predictions.values, label='이동평균 예측', color='green', linestyle='--')
plt.title(f'이동평균 모델 (Window={WINDOW_SIZE})')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_이동평균모델.png')
plt.show()

# 모델 평가
ma_mse = mean_squared_error(test_data, ma_predictions)
ma_rmse = np.sqrt(ma_mse)
ma_mae = mean_absolute_error(test_data, ma_predictions)
ma_r2 = r2_score(test_data, ma_predictions)

print("이동평균 모델 평가:")
print(f"RMSE: {ma_rmse:.4f}")
print(f"MAE: {ma_mae:.4f}")
print(f"R²: {ma_r2:.4f}")

### 계절성 모델 (Seasonal Naive)
계절성 모델은 과거의 동일한 계절(주기)에 해당하는 값을 사용하여 미래를 예측합니다.
이 모델은 데이터에 뚜렷한 계절적 패턴이 있을 때 특히 유용합니다.

**핵심 아이디어**: 특정 시점의 값은 한 계절 이전의 같은 시점 값과 유사할 것이라는 가정

아래 코드에서는 `WINDOW_SIZE`를 계절성 주기로 사용합니다.
테스트 데이터의 각 시점에 대해, 해당 시점보다 정확히 `WINDOW_SIZE` 만큼 이전의 값을 예측값으로 사용합니다.
이전 계절 데이터가 충분하지 않은 경우에는 가장 최근 관측값을 사용합니다.

In [ ]:
# 계절성 모델 (이전 계절의 값 사용)
seasonal_predictions = []

# 테스트 데이터의 각 시점에 대해 계절성 주기에 해당하는 이전 값 사용
for i in range(len(test_data)):
    # 현재 테스트 데이터 위치에서 계절성 주기만큼 이전 시점의 인덱스 계산
    seasonal_idx = len(train_data) + i - WINDOW_SIZE
    
    # 계산된 인덱스가 유효한 훈련 데이터 범위 내인지 확인
    if 0 <= seasonal_idx < len(train_data):
        # 유효한 위치면 해당 과거 값 사용
        seasonal_value = train_data.iloc[seasonal_idx]
    else:
        # 적절한 이전 값이 없으면 훈련 데이터의 마지막 값 사용
        seasonal_value = train_data.iloc[-1]
    
    seasonal_predictions.append(seasonal_value)

# 예측값을 테스트 데이터와 같은 인덱스를 가진 시리즈로 변환
seasonal_predictions = pd.Series(seasonal_predictions, index=test_data.index)

# 시각화
plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data.values, label='훈련 데이터', color='blue')
plt.plot(test_data.index, test_data.values, label='실제 테스트 데이터', color='red')
plt.plot(seasonal_predictions.index, seasonal_predictions.values, label='계절성 예측', color='green', linestyle='--')
plt.title(f'계절성 모델 (주기={WINDOW_SIZE})')
plt.xlabel('날짜')
plt.ylabel(TARGET_COL)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_계절성모델.png')
plt.show()

# 모델 평가
seasonal_mse = mean_squared_error(test_data, seasonal_predictions)
seasonal_rmse = np.sqrt(seasonal_mse)
seasonal_mae = mean_absolute_error(test_data, seasonal_predictions)
seasonal_r2 = r2_score(test_data, seasonal_predictions)

print("계절성 모델 평가:")
print(f"RMSE: {seasonal_rmse:.4f}")
print(f"MAE: {seasonal_mae:.4f}")
print(f"R²: {seasonal_r2:.4f}")

## 11. 모델 비교
여러 기본 모델들의 성능 지표(RMSE, MAE, R²)를 비교하여 어떤 단순 모델이 해당 데이터에 상대적으로 더 적합한지 평가합니다.
이를 통해 데이터의 기본적인 특징(예: 추세 존재 여부, 계절성 강도)을 추론할 수 있습니다.

In [ ]:
# 모델 성능 비교 - 모든 모델 결과를 한 테이블에 정리
models = ['추세 없음', '선형 추세', '이동평균', '계절성']
rmse_scores = [naive_rmse, linear_rmse, ma_rmse, seasonal_rmse]
mae_scores = [naive_mae, linear_mae, ma_mae, seasonal_mae]
r2_scores = [naive_r2, linear_r2, ma_r2, seasonal_r2]

# 성능 비교 데이터프레임 생성
performance_df = pd.DataFrame({
    '모델': models,
    'RMSE': rmse_scores,
    'MAE': mae_scores,
    'R²': r2_scores
})

# 성능 순위 계산 (RMSE 기준으로 오름차순 정렬)
performance_df_sorted = performance_df.sort_values('RMSE')

print("\n모델 성능 비교 (RMSE 기준 오름차순):")
print(performance_df_sorted)

# 최고 성능 모델 찾기 (RMSE 기준)
best_model_idx = np.argmin(rmse_scores)
best_model = models[best_model_idx]
best_rmse = rmse_scores[best_model_idx]
best_mae = mae_scores[best_model_idx]
best_r2 = r2_scores[best_model_idx]

# 성능 비교 시각화
plt.figure(figsize=(16, 10))

plt.subplot(2, 2, 1)
plt.bar(models, rmse_scores)
plt.title('RMSE 비교 (낮을수록 좋음)')
plt.ylabel('RMSE')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.bar(models, mae_scores)
plt.title('MAE 비교 (낮을수록 좋음)')
plt.ylabel('MAE')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.bar(models, r2_scores)
plt.title('R² 비교 (1에 가까울수록 좋음)')
plt.ylabel('R²')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/basic_모델성능비교.png')
plt.show()

## 12. 결론 및 다음 단계
시계열 분석의 기본 단계를 수행하여 데이터의 특성을 파악하고 간단한 모델을 적용하여 기본적인 예측 성능을 확인했습니다.
이를 통해 데이터의 특성(추세, 계절성, 정상성 등)과 간단한 모델의 성능을 파악할 수 있었습니다.

**분석 결론**:
- 데이터 특성: 시계열 분해와 통계적 검정을 통해 데이터의 추세, 계절성, 정상성 등의 특성을 파악했습니다.
- 기본 모델 성능: 여러 간단한 모델(추세 없음, 선형 추세, 이동평균, 계절성) 중 데이터에 가장 적합한 모델을 확인했습니다.

**다음 단계**:
- 고급 시계열 모델 적용: ARIMA, SARIMA, 지수평활법(ETS) 등의 통계적 모델 적용
- 외부 변수 활용: 외부 요인을 고려한 모델링(SARIMAX, 회귀 모델 등)
- 머신러닝 기반 모델: RandomForest, XGBoost 등의 머신러닝 모델을 시계열 특성과 함께 활용
- 딥러닝 모델: LSTM, GRU 등의 순환 신경망 모델 적용